In [ ]:
import SimpleITK as sitk
import numpy as np
import json
import cv2
import copy
from tqdm import tqdm
from tqdm.contrib import tzip
import os
import numpy as np
from glob import glob
import skimage.io as io
import warnings
warnings.filterwarnings('ignore')

In [ ]:

# #read the image
# img = cv2.imread("hospital_data_clean/png/origin/id01_0.png")
# gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
# # cv2.namedWindow('original',cv2.WINDOW_AUTOSIZE)
# # cv2.imshow('original',gray)
 
# #take the  threshold and morphology thransform
# ret, binary = cv2.threshold(gray, 30, 255, cv2.THRESH_BINARY)
# kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(2,2))
# bin_clo = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel, iterations = 2)

# cv2.imwrite('1.png',bin_clo)

# #obtain th label of the connection areas
# num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(bin_clo,connectivity = 8)

# 转成int8就行

# reader = sitk.ImageFileReader()
# reader.SetFileName(path_rebi)
# image = reader.Execute()
# img_array = sitk.GetArrayFromImage(image)

# img_array = img_array.astype(np.int8)
# img_array = img_array.reshape(512,512)
# print(check_item(img_array))
# print(img_array.shape)
# num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(img_array,connectivity = 8)

In [ ]:
def check_item(arr):
    a = dict()
    for i in tqdm(arr.reshape(-1)):
        try:
            a[i] += 1
        except:
            a[i] = 1
    
    return a

In [ ]:
# 删除中上部分
def del_body(arr,connects=8):
    
    '''
    删除CT图中上部分的联通区域。
    arr:待处理矩阵
    connects:连通数
    '''
    
    # step1 确定中轴线，若偏差过大则定义为256
    ind_list = []
    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(arr,connectivity = connects)
    
    if num_labels == 1:
        return arr,0
    
    stats_temp = stats[1::]
    up = stats_temp[:,1].min()
        
    down = (stats_temp[:,1] + stats_temp[:,3]).max()

    left = stats_temp[:,0].min()
    right = (stats_temp[:,0] + stats_temp[:,2]).max()

    y_mid = (up + down)/2
    x_mid = (left + right)/2
    
    if abs(x_mid-256)>=15:
        if x_mid > 256:
            x_mid = 256 + abs(x_mid-256)/3
        else:
            x_mid = 256 - abs(x_mid-256)/3
            
    if abs(y_mid-256)>=15:
        if y_mid > 256:
            y_mid = 256 + abs(y_mid-256)/3
        else:
            y_mid = 256 - abs(y_mid-256)/3    
    
    # step2 删除质心的x在纵中轴线+-100，y在横向中轴线以上的所有连通区域。
    for i in range(1,stats.shape[0]):
        if centroids[i,1] < y_mid and centroids[i,0] >= x_mid-100 and centroids[i,0] <= x_mid+100:
            ind_list.append(i)
    
    for ind in ind_list:
        arr[labels==ind] = 0
    
    return arr,1

In [ ]:
# 删除面积小于area的连通域操作
def del_pix(arr,area,connects=8):
    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(arr,connectivity = connects)
    index_del = []
    
    for i in range(stats.shape[0]):
        if stats[i][4] <= area:
            index_del.append(i)
        
    arr = arr.astype(np.uint8)
    
    for ind in index_del:
        arr[labels==ind] = 0
        
    return arr

In [ ]:
# 删除扫描床
def del_bed(arr,connects=8):
    '''
    判断质心在在最底下且长度大于120，或者质心和最大值相差30的距离，并且长度大于120
    
    '''
    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(arr,connectivity = connects)
    del_list = []
    for i in range(1,stats.shape[0]):
        if centroids[:,1].max() == centroids[i,1] and stats[i,2] >= 120:
            del_list.append(i)
        elif abs(centroids[:,1].max() - centroids[i,1]) <= 30 and stats[i,2] >= 120:
            del_list.append(i)

    for ind in del_list:
        arr[labels==ind]=0
        
    return arr

In [ ]:
# 去除除骨头以外部分
def del_area(arr,area=10,flag=1,connect=8):
    '''
    arr:需要切除的矩阵
    area:第一步中去除的小像素值
    flag:是否进行与操作
    connect:连通域选择4/8 一般选8
    
    '''
    res = np.copy(arr)
    
    # step1 清理扫描床
    arr = del_bed(arr,connects=connect)
    
    # step2 清理小pix,处理总像素小于等于area的连通块
    arr = del_pix(arr,area=area,connects=connect)
    
    # step3 闭操作
    kernel = np.ones((5,5),np.uint8)
    # opening = cv2.morphologyEx(res.astype(np.uint8), cv2.MORPH_OPEN, np.ones((2,2),np.uint8))
    # io.imsave('A_kai2.png',opening.astype(np.uint8))    # 开操作
    arr1 = cv2.morphologyEx(arr.astype(np.uint8), cv2.MORPH_CLOSE, kernel) # 闭操作
    
    # step4 清理中上部分
    arr,check_num = del_body(arr1,connects=connect)
    
    if check_num == 0:
        arr = arr1
    
    # step5 逻辑“与”运算
    if flag:
        arr [np.logical_and(arr,res)==0] = 0
    
    return arr

In [26]:
# 做论文测试图
path_rebi = 'hospital_data_clean/dcm/rebi_mask/id12_192.dcm'
path_mask = 'hospital_data_clean/dcm/mask/id12_192.dcm'
mask = io.imread(path_mask,3)
rebi = io.imread(path_rebi,3)

connect = 8
flag = 1

mask[mask==1] = 255
arr = rebi - mask
io.imsave('A_1_origin.png',arr.astype(np.uint8))
res = np.copy(arr)

# step1 检查扫描床
arr = del_bed(arr,connects=connect)
io.imsave('A_2_delbed.png',arr.astype(np.uint8))

# step2 清理小pix,处理总像素小于等于area的连通块
arr = del_pix(arr,area = 10,connects=connect)
io.imsave('A_3_delpix.png',arr.astype(np.uint8))

# step3 闭操作
kernel = np.ones((5,5),np.uint8)
arr = cv2.morphologyEx(arr.astype(np.uint8), cv2.MORPH_CLOSE, kernel) # 闭操作
io.imsave('A_4_close.png',arr.astype(np.uint8))

# step4 清理中上部分
arr,_ = del_body(arr,connects=connect)
io.imsave('A_5_delbody.png',arr.astype(np.uint8))

# step5 逻辑“与”运算
if flag:
    arr [np.logical_and(arr,res)==0] = 0
    
    io.imsave('A_6_and.png',arr.astype(np.uint8))


In [ ]:
# 将测试volumn变成nii文件
# 看效果用transfer2nii的文件
path_ves = 'hospital_testvolumn/mask/'
path_all = 'hospital_testvolumn/rebi_mask/'
path_out = 'hospital_testvolumn/mask_bone/'
name = os.listdir(path_ves)
name.sort(key=lambda x:int(x.split('_')[1].split('.')[0]))

for item in tqdm(name):
    mask = io.imread(path_ves+item,3)
    rebi = io.imread(path_all+item,3)

    mask[mask==1] = 255
    res = rebi - mask
    
    arr = del_area(res,area = 10,flag = 1,connect = 8)
    io.imsave(path_out+item,arr)


## rebi_mask/origin/mask 转nii
import sys,os
import pandas as pd

from tools import fold2nii
for name in ['mask_bone']:
    path_ = 'hospital_testvolumn/{0}'.format(name)
    q = os.listdir(path_)
    q.sort(key=lambda x:int(x.split('_')[1].split('.')[0]))
    d = path_ + '/' + pd.Series(q) 
    d = d.to_list()
    fold2nii(d,path_)

In [ ]:
# 将hospital_data_clean 里dcm里的mask和rebi_mask提取并制作真正的mask
path_ves = 'hospital_data_clean/dcm/mask/'
path_all = 'hospital_data_clean/dcm/rebi_mask/'
path_out = 'hospital_data_clean/png/mask_bone/'

name = os.listdir(path_ves)
name.sort(key=lambda x:int(x.split('_')[1].split('.')[0]))

for item in tqdm(name):
    mask = io.imread(path_ves+item,3)
    rebi = io.imread(path_all+item,3)

    mask[mask==1] = 255
    res = rebi - mask
    
    arr = del_area(res,10,1,8)
    io.imsave(path_out+item.replace('dcm','png'),arr)
